In [106]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv


# Reading dataset

In [107]:
df = pd.read_csv("/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv")

# Visualizing Data

In [108]:
#Defining function for Continuous and catogorical variable
def plot_conti(x):
    fig, axes = plt.subplots(nrows=1,ncols=3,figsize=(15,5),tight_layout=True)
    axes[0].set_title('Histogram')
    sns.histplot(x,ax=axes[0])
    axes[1].set_title('Checking Outliers')
    sns.boxplot(x,ax=axes[1])
    axes[2].set_title('Relation with output variable')
    sns.boxplot(y = x,x = df['Fertilizer Name'])
    
def plot_cato(x):
    fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(15,5),tight_layout=True)
    axes[0].set_title('Count Plot')
    sns.countplot(x,ax=axes[0])
    axes[1].set_title('Relation with output variable')
    sns.countplot(x = x,hue = df['Fertilizer Name'], ax=axes[1])

# Preprocessing using One-Hot Encoder

In [109]:
y = df['Fertilizer Name'].copy()
X = df.drop(['Fertilizer Name','Soil Type'], axis=1).copy()



In [110]:
X

,Temparature,Humidity,Moisture,Crop Type,Nitrogen,Potassium,Phosphorous
0,26,52,38,Maize,37,0,0
1,29,52,45,Sugarcane,12,0,36
2,34,65,62,Cotton,7,9,30
3,32,62,34,Tobacco,22,0,20
4,28,54,46,Paddy,35,0,0
...,...,...,...,...,...,...,...
94,25,50,32,Pulses,24,0,19
95,30,60,27,Tobacco,4,17,17
96,38,72,51,Wheat,39,0,0
97,36,60,43,Millets,15,0,41


In [111]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [112]:
X

array([[ 0.,  0.,  0., ..., 37.,  0.,  0.],
       [ 0.,  0.,  0., ..., 12.,  0., 36.],
       [ 0.,  1.,  0., ...,  7.,  9., 30.],
       ...,
       [ 0.,  0.,  0., ..., 39.,  0.,  0.],
       [ 0.,  0.,  0., ..., 15.,  0., 41.],
       [ 0.,  0.,  0., ..., 12.,  0., 10.]])

In [113]:
print(np.size(X))

1683


# Train-test split

In [114]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=42)

# Feature Scaling

In [115]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [116]:
X_test

array([[-0.24806947,  2.97609524, -0.33601075, -0.17277369, -0.36214298,
        -0.24806947, -0.36214298, -0.3086067 , -0.43549417, -0.24806947,
        -0.36214298,  1.06134936,  0.9915576 ,  1.86217313,  0.46072126,
        -0.57643157,  0.16743993],
       [-0.24806947, -0.33601075, -0.33601075, -0.17277369, -0.36214298,
        -0.24806947, -0.36214298,  3.24037035, -0.43549417, -0.24806947,
        -0.36214298, -0.93332667, -0.86669479, -1.18643689, -0.49732651,
        -0.57643157, -0.36814607],
       [-0.24806947, -0.33601075, -0.33601075, -0.17277369, -0.36214298,
        -0.24806947, -0.36214298, -0.3086067 , -0.43549417,  4.03112887,
        -0.36214298, -0.07846552,  0.14689742, -1.4554319 , -1.28118377,
         2.51103066, -0.06209693],
       [-0.24806947,  2.97609524, -0.33601075, -0.17277369, -0.36214298,
        -0.24806947, -0.36214298, -0.3086067 , -0.43549417, -0.24806947,
        -0.36214298, -0.64837295, -0.86669479,  1.95183813,  1.76715002,
        -0.57643157

In [117]:
X_train[0]

array([-0.24806947, -0.33601075, -0.33601075, -0.17277369, -0.36214298,
       -0.24806947, -0.36214298,  3.24037035, -0.43549417, -0.24806947,
       -0.36214298, -1.50323411, -1.54242294, -1.00710689,  0.46072126,
       -0.57643157,  0.09092764])

# Random Forest Classifier

In [118]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators= 100, criterion = 'gini' , random_state= 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [119]:
y_pred = classifier.predict(X_test)

In [135]:
X_test

array([[-0.24806947,  2.97609524, -0.33601075, -0.17277369, -0.36214298,
        -0.24806947, -0.36214298, -0.3086067 , -0.43549417, -0.24806947,
        -0.36214298,  1.06134936,  0.9915576 ,  1.86217313,  0.46072126,
        -0.57643157,  0.16743993],
       [-0.24806947, -0.33601075, -0.33601075, -0.17277369, -0.36214298,
        -0.24806947, -0.36214298,  3.24037035, -0.43549417, -0.24806947,
        -0.36214298, -0.93332667, -0.86669479, -1.18643689, -0.49732651,
        -0.57643157, -0.36814607],
       [-0.24806947, -0.33601075, -0.33601075, -0.17277369, -0.36214298,
        -0.24806947, -0.36214298, -0.3086067 , -0.43549417,  4.03112887,
        -0.36214298, -0.07846552,  0.14689742, -1.4554319 , -1.28118377,
         2.51103066, -0.06209693],
       [-0.24806947,  2.97609524, -0.33601075, -0.17277369, -0.36214298,
        -0.24806947, -0.36214298, -0.3086067 , -0.43549417, -0.24806947,
        -0.36214298, -0.64837295, -0.86669479,  1.95183813,  1.76715002,
        -0.57643157

# Creating confusion matrix

In [120]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[3 0 0 0 0 0 0]
 [0 4 0 0 0 0 0]
 [0 0 2 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 6 0 0]
 [0 0 0 0 0 7 0]
 [0 0 0 0 0 0 7]]


1.0

In [121]:
classifier.score(X_test, y_test)

1.0

**Test accuracy = 96.67%**

# Preprocessing using Label Encoder

In [122]:
#encoding the labels for categorical variables
from sklearn.preprocessing import LabelEncoder


In [123]:
#encoding Soil Type variable
encode_soil = LabelEncoder()
df['Soil Type'] = encode_soil.fit_transform(df['Soil Type'])

#creating the DataFrame
Soil_Type = pd.DataFrame(zip(encode_soil.classes_,encode_soil.transform(encode_soil.classes_)),columns=['Original','Encoded'])
Soil_Type = Soil_Type.set_index('Original')
Soil_Type

,Encoded
Original,
Black,0
Clayey,1
Loamy,2
Red,3
Sandy,4


In [124]:
encode_crop =  LabelEncoder()
df['Crop Type'] = encode_crop.fit_transform(df['Crop Type'])

#creating the DataFrame
Crop_Type = pd.DataFrame(zip(encode_crop.classes_,encode_crop.transform(encode_crop.classes_)),columns=['Original','Encoded'])
Crop_Type = Crop_Type.set_index('Original')
Crop_Type

,Encoded
Original,
Barley,0
Cotton,1
Ground Nuts,2
Maize,3
Millets,4
Oil seeds,5
Paddy,6
Pulses,7
Sugarcane,8


In [125]:
encode_ferti = LabelEncoder()
df['Fertilizer Name'] = encode_ferti.fit_transform(df['Fertilizer Name'])

#creating the DataFrame
Fertilizer = pd.DataFrame(zip(encode_ferti.classes_,encode_ferti.transform(encode_ferti.classes_)),columns=['Original','Encoded'])
Fertilizer = Fertilizer.set_index('Original')
Fertilizer

,Encoded
Original,
10-26-26,0
14-35-14,1
17-17-17,2
20-20,3
28-28,4
DAP,5
Urea,6


In [126]:
#splitting the data into train and test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.drop('Fertilizer Name',axis=1),df['Fertilizer Name'],test_size=0.2,random_state=1)
print('Shape of Splitting :')
print('x_train = {}, y_train = {}, x_test = {}, y_test = {}'.format(x_train.shape,y_train.shape,x_test.shape,y_test.shape))

Shape of Splitting :
x_train = (79, 8), y_train = (79,), x_test = (20, 8), y_test = (20,)


In [127]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79 entries, 2 to 37
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Temparature  79 non-null     int64
 1   Humidity     79 non-null     int64
 2   Moisture     79 non-null     int64
 3   Soil Type    79 non-null     int64
 4   Crop Type    79 non-null     int64
 5   Nitrogen     79 non-null     int64
 6   Potassium    79 non-null     int64
 7   Phosphorous  79 non-null     int64
dtypes: int64(8)
memory usage: 5.6 KB


# Random Forest Classifier

In [128]:
rand = RandomForestClassifier(random_state = 42)
rand.fit(x_train,y_train)

RandomForestClassifier(random_state=42)

In [129]:
pred_rand = rand.predict(x_test)

# Hyperparameter tuning with GridSearchCV

The performance of a model significantly depends on the value of hyperparameters. Grid Search is the process of performing hyperparameter tuning in order to determine the optimal values for a given model. 

In [130]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

params = {
    'n_estimators':[300,400,500],
    'max_depth':[5,10,15],
    'min_samples_split':[2,5,8]
}
grid_rand = GridSearchCV(rand,params,cv=3,verbose=3,n_jobs=-1)

grid_rand.fit(x_train,y_train)

pred_rand = grid_rand.predict(x_test)

print(classification_report(y_test,pred_rand))

print('Best score : ',grid_rand.best_score_)
print('Best params : ',grid_rand.best_params_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           1       0.75      1.00      0.86         3
           2       0.67      1.00      0.80         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         6

    accuracy                           0.90        20
   macro avg       0.92      0.90      0.88        20
weighted avg       0.93      0.90      0.88        20

Best score :  0.9748338081671415
Best params :  {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 300}


**Best score = 97.48%**

In [131]:
y_train[2]

1

In [132]:
#pickling the file
import pickle
pickle_out = open('classifier.pkl','wb')
pickle.dump(grid_rand,pickle_out)
pickle_out.close()

In [133]:
df.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,26,52,38,4,3,37,0,0,6
1,29,52,45,2,8,12,0,36,5
2,34,65,62,0,1,7,9,30,1
3,32,62,34,3,9,22,0,20,4
4,28,54,46,1,6,35,0,0,6


In [134]:
model = pickle.load(open('classifier.pkl','rb'))
ans = model.predict([[34,65,62	,0,	1,	7,	9,	30]])
if ans[0] == 0:
    print("10-26-26")
elif ans[0] ==1:
    print("14-35-14")
elif ans[0] == 2:
    print("17-17-17	")
elif ans[0] == 3:
    print("20-20")
elif ans[0] == 4:
    print("28-28")
elif ans[0] == 5:
    print("DAP")
else:
    print("Urea")

14-35-14
